In [1]:
import scrapy
import requests
from scrapy.http import TextResponse

In [ ]:
# 1. Start project

In [2]:
!scrapy startproject daangn

New Scrapy project 'daangn', using template directory '/opt/anaconda3/lib/python3.7/site-packages/scrapy/templates/project', created in:
    /Users/fofx/dss15/crawling project/daangn

You can start your first spider with:
    cd daangn
    scrapy genspider example example.com


In [ ]:
# 2. Check xpath
# get item link data > get detail info

In [2]:
# request to get item link
query = 'macbook'
url = 'https://www.daangn.com/search/{}'.format(query)
response = requests.get(url)
response

<Response [200]>

In [3]:
dom = TextResponse(response.url, body=response.text, encoding="utf-8")
links = dom.xpath('//*[@id="flea-market-wrap"]/article/a/@href').extract()
len(links), links[0]

(6, '/articles/150586122')

In [4]:
# get full url
dom.urljoin(links[0])
link = dom.urljoin(links[0])

In [5]:
# request link
response = requests.get(link)
dom = TextResponse(response.url, body=response.text, encoding="utf-8")

In [6]:
# get xpath for each element
category = dom.xpath('//*[@id="article-category"]/text()')[0].extract().replace('∙', '').strip()
title = dom.xpath('//*[@id="article-title"]/text()')[0].extract().strip()
user_id = dom.xpath('//*[@id="nickname"]/text()')[0].extract().strip()
region = dom.xpath('//*[@id="region-name"]/text()')[0].extract().strip()
counts = dom.xpath('//*[@id="article-counts"]/text()')[0].extract().strip()
chat_counts = counts.split('∙')[0].strip().split(' ')[1]
watch_counts = counts.split('∙')[1].strip().split(' ')[1]
view_counts = counts.split('∙')[2].strip().split(' ')[1]
desc = dom.xpath('//*[@id="article-detail"]/p/text()').extract()
desc = "".join(desc).replace('\n', '')
price = dom.xpath('//*[@id="article-price"]/text()')[0].extract().strip()


category, title, price, region, user_id, desc, chat_counts, watch_counts, view_counts

('디지털/가전',
 '맥북에어 (MacBook Air) 11인치 Mid 2011',
 '250,000원',
 '강서구 명지동',
 '하민캐롯',
 '- 상품명 : MacBook Air 11인치 Mid 2011- CPU : 1.6 GHz Intel Core i5- RAM : 4GB (DDR3)- SSD : 120GB- 외관상태 : A급 (흠집 없음)- 맥os Sierra ➡️ El Captain (초기화 완료)- 구매 후, 설정 입력해서 사용하면 됨✅- 맥북에어 전용 파우치⭕️+전원어댑터⭕️- 맥북에어 박스❌',
 '2',
 '6',
 '162')

In [ ]:
# 3. items.py

In [65]:
# %load daangn/daangn/items.py

In [84]:
%%writefile daangn/daangn/items.py
import scrapy


class DaangnItem(scrapy.Item):
    category = scrapy.Field() 
    title = scrapy.Field() 
    price = scrapy.Field() 
    region = scrapy.Field() 
    user_id = scrapy.Field() 
    desc = scrapy.Field() 
    chat_counts = scrapy.Field() 
    watch_counts = scrapy.Field() 
    view_counts = scrapy.Field()

Overwriting daangn/daangn/items.py


In [ ]:
# 4. spider.py

In [7]:
%%writefile daangn/daangn/spiders/spider.py
import scrapy
from daangn.items import DaangnItem

class DaangnSpider(scrapy.Spider):
    name = "Daangn"
    allow_domain = ["daangn.com"]
    start_urls = ["https://www.daangn.com/search/macbook"]
    
    def parse(self, response):
        links = dom.xpath('//*[@id="flea-market-wrap"]/article/a/@href').extract()
        links = dom.urljoin(links)
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = DaangnItem()
        item['category'] = dom.xpath('//*[@id="article-category"]/text()')[0].extract().replace('∙', '').strip()
        item['title'] = dom.xpath('//*[@id="article-title"]/text()')[0].extract().strip()
        item['price'] = dom.xpath('//*[@id="article-price"]/text()')[0].extract().strip()
        desc = dom.xpath('//*[@id="article-detail"]/p/text()').extract()
        item['desc'] = "".join(desc).replace('\n', '')
        item['user_id'] = dom.xpath('//*[@id="nickname"]/text()')[0].extract().strip()
        item['region'] = dom.xpath('//*[@id="region-name"]/text()')[0].extract().strip()
        counts = dom.xpath('//*[@id="article-counts"]/text()')[0].extract().strip()
        item['chat_counts'] = counts.split('∙')[0].strip().split(' ')[1]
        item['watch_counts'] = counts.split('∙')[1].strip().split(' ')[1]
        item['view_counts'] = counts.split('∙')[2].strip().split(' ')[1]
       
        yield item

Overwriting daangn/daangn/spiders/spider.py


In [86]:
!tree daangn/

daangn/
├── daangn
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-37.pyc
│   │   ├── items.cpython-37.pyc
│   │   └── settings.cpython-37.pyc
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       ├── __pycache__
│       │   ├── __init__.cpython-37.pyc
│       │   └── spider.cpython-37.pyc
│       └── spider.py
├── items.csv
└── scrapy.cfg

4 directories, 14 files


In [8]:
%%writefile run.sh
cd daangn
rm items.csv
scrapy crawl Daangn -o items.csv

Overwriting run.sh


In [9]:
!/bin/bash run.sh

2020-11-17 23:41:28 [scrapy.utils.log] INFO: Scrapy 2.4.0 started (bot: daangn)
2020-11-17 23:41:28 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.6 (default, Jan  8 2020, 13:42:34) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Darwin-19.6.0-x86_64-i386-64bit
2020-11-17 23:41:28 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-11-17 23:41:28 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'daangn',
 'NEWSPIDER_MODULE': 'daangn.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['daangn.spiders']}
2020-11-17 23:41:28 [scrapy.extensions.telnet] INFO: Telnet Password: 6d11c8c9035e5ff7
2020-11-17 23:41:28 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 's

In [87]:
import pandas as pd

In [88]:
# df = pd.read_csv("daangn/items.csv")

EmptyDataError: No columns to parse from file